In [1]:
import numpy as np 
import networkx as nx
from matplotlib import pyplot as plt

In [2]:
# Defining some utilities (early implementation, will be refactored in a class)

def randomEnvironment(n: int,
                      d: int):
    
    return np.random.multivariate_normal(np.zeros(d),
                                         np.eye(d),
                                         n)

def distanceBasedPermutation(x: np.array, 
                             ord = 2):
    
    DBP = np.zeros((x.shape[0],x.shape[0]))
    for i in range(x.shape[0]):
        for j in range(x.shape[0]):
            DBP[i,j] = np.linalg.norm(x[i] - x[j], ord)

    return np.argsort(DBP, 1)

def randomizedConstruction(x: np.array,
                           ord = 2):

    n = x.shape[0]
    A = np.zeros((n,n))

    DBP = distanceBasedPermutation(x, ord)
    m = int(np.floor(np.sqrt(n * np.log(n))))

    for i in range(n):

        # Deterministic step
        A[i, DBP[i, 1:m]] = 1
        A[DBP[i,1:m], i]
        # Randomized step
        S = int(np.ceil(3 * np.sqrt(n * np.log(n))))
        R = np.random.choice(np.concatenate([np.arange(0,i), np.arange(i+1,n)]), S)

        A[i, R] = 1
        A[R, i] = 1

    return A

In [3]:
X = randomEnvironment(100, 5)

In [4]:
randomizedConstruction(X)

array([[0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [0., 1., 1., ..., 0., 1., 0.],
       [1., 1., 1., ..., 1., 0., 1.],
       [1., 1., 1., ..., 1., 1., 0.]])

In [5]:
A = randomizedConstruction(X)
G = nx.from_numpy_matrix(A)

In [6]:
nx.is_connected(G)

True

_________

In [7]:
# Greedy routing algorithm

def greedyRouting(s: int,
                  t: int,
                  x: np.array,
                  ):
    
    A = randomizedConstruction(x)

    done = False
    j = s

    steps = 0

    while not done:
        
        if np.all(A[j] == 0):
            done = True
            print(j)

        else:
            neighs = np.where(A[j] == 1)[0]
            X_ = np.copy(x[neighs])
            h = neighs[np.argmin(np.linalg.norm(X_ - x[t], axis = 1))]

            if np.linalg.norm(x[t] - x[h]) < np.linalg.norm(x[t] - x[j]):
                j = h
                steps += 1
                
            else:
                done = True


    return x[j], A, steps

In [8]:
X = randomEnvironment(1000, 5)

X1, B, steps = greedyRouting(0, 50, X)

In [9]:
np.all(X1 == X[50])

True

In [11]:
steps <= 2

True